In [ ]:
from ortools.sat.python import cp_model
import pandas as pd

df = pd.read_csv('./data/docente_classes.csv')
teachers = df['Identificativo'].dropna().unique()
days = ['LUN', 'MAR', 'MER', 'GIO', 'VEN']
hours = [8, 9, 10, 11, 12, 13, 14]

model = cp_model.CpModel()
assign = {}

# Variables: assign[(teacher, class, day, hour)] = 1 if assigned, else 0
for _, row in df.iterrows():
    teacher = row['Identificativo']
    class_ = row['Classi']
    n_ore = row['N.Ore']
    if pd.isnull(teacher) or pd.isnull(class_) or pd.isnull(n_ore):
        continue
    for day in days:
        for hour in hours:
            assign[(teacher, class_, day, hour)] = model.NewBoolVar(f'{teacher}_{class_}_{day}{hour}')

# Constraints:
# 1. Each teacher/class gets required hours
for _, row in df.iterrows():
    teacher = row['Identificativo']
    class_ = row['Classi']
    n_ore = row['N.Ore']
    if pd.isnull(teacher) or pd.isnull(class_) or pd.isnull(n_ore):
        continue
    model.Add(
        sum(assign[(teacher, class_, day, hour)] for day in days for hour in hours) == int(n_ore)
    )

# 2. No double-booking for teachers or classes
for teacher in teachers:
    for day in days:
        for hour in hours:
            model.Add(
                sum(assign.get((teacher, class_, day, hour), 0) for class_ in df[df['Identificativo'] == teacher]['Classi']) <= 1
            )
for class_ in df['Classi'].dropna().unique():
    for day in days:
        for hour in hours:
            model.Add(
                sum(assign.get((teacher, class_, day, hour), 0) for teacher in df[df['Classi'] == class_]['Identificativo']) <= 1
            )

# 3. Max 5, min 2 hours per day per teacher
# for teacher in teachers:
#     for day in days:
#         model.Add(
#             sum(assign.get((teacher, class_, day, hour), 0)
#                 for class_ in df[df['Identificativo'] == teacher]['Classi']
#                 for hour in hours) <= 5
#         )
#         model.Add(
#             sum(assign.get((teacher, class_, day, hour), 0)
#                 for class_ in df[df['Identificativo'] == teacher]['Classi']
#                 for hour in hours) >= 2
#         )

# 4. (Advanced) Holes constraints: requires more logic, e.g., introduce auxiliary variables for holes.

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Extract solution
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Build schedule DataFrame
    schedule = pd.DataFrame('', index=teachers, columns=[f'{d}{h}' for d in days for h in hours])
    for key, var in assign.items():
        if solver.Value(var):
            teacher, class_, day, hour = key
            schedule.loc[teacher, f'{day}{hour}'] = class_
    schedule.to_csv('generated_schedule_ortools.csv')
else:
    # # Build schedule DataFrame
    # schedule = pd.DataFrame('', index=teachers, columns=[f'{d}{h}' for d in days for h in hours])
    # for key, var in assign.items():
    #     if solver.Value(var):
    #         teacher, class_, day, hour = key
    #         schedule.loc[teacher, f'{day}{hour}'] = class_
    # schedule.to_csv('generated_schedule_ortools.csv')
    print('No solution found.')

No solution found.


In [4]:
for class_ in df['Classi'].dropna().unique():
    for day in days:
        for hour in hours:
            print(sum(assign.get((teacher, class_, day, hour), 0) for teacher in df[df['Classi'] == class_]['Identificativo']) <= 1)


(ALLEGRA P._1E_LUN8 + ARMETTA R._1E_LUN8 + BONDI G._1E_LUN8 + CAVADI A._1E_LUN8 + COLOMBINO E._1E_LUN8 + MACCATAIO S._1E_LUN8 + MONTALBANO B._1E_LUN8 + NICCHITTA D._1E_LUN8 + PASSARO G._1E_LUN8 + PATERNO A._1E_LUN8 + 2 * RICCOBONO A._1E_LUN8 + SORCI A._1E_LUN8) <= 1
(ALLEGRA P._1E_LUN9 + ARMETTA R._1E_LUN9 + BONDI G._1E_LUN9 + CAVADI A._1E_LUN9 + COLOMBINO E._1E_LUN9 + MACCATAIO S._1E_LUN9 + MONTALBANO B._1E_LUN9 + NICCHITTA D._1E_LUN9 + PASSARO G._1E_LUN9 + PATERNO A._1E_LUN9 + 2 * RICCOBONO A._1E_LUN9 + SORCI A._1E_LUN9) <= 1
(ALLEGRA P._1E_LUN10 + ARMETTA R._1E_LUN10 + BONDI G._1E_LUN10 + CAVADI A._1E_LUN10 + COLOMBINO E._1E_LUN10 + MACCATAIO S._1E_LUN10 + MONTALBANO B._1E_LUN10 + NICCHITTA D._1E_LUN10 + PASSARO G._1E_LUN10 + PATERNO A._1E_LUN10 + 2 * RICCOBONO A._1E_LUN10 + SORCI A._1E_LUN10) <= 1
(ALLEGRA P._1E_LUN11 + ARMETTA R._1E_LUN11 + BONDI G._1E_LUN11 + CAVADI A._1E_LUN11 + COLOMBINO E._1E_LUN11 + MACCATAIO S._1E_LUN11 + MONTALBANO B._1E_LUN11 + NICCHITTA D._1E_LUN11 + PASS

In [8]:
for class_ in df['Classi'].dropna().unique():
    for day in days:
        for hour in hours:
            for teacher in df[df['Classi'] == class_]['Identificativo']:
                print(assign.get((teacher, class_, day, hour), 0))

ALLEGRA P._1E_LUN8
ARMETTA R._1E_LUN8
BONDI G._1E_LUN8
CAVADI A._1E_LUN8
COLOMBINO E._1E_LUN8
MACCATAIO S._1E_LUN8
MONTALBANO B._1E_LUN8
NICCHITTA D._1E_LUN8
PASSARO G._1E_LUN8
PATERNO A._1E_LUN8
RICCOBONO A._1E_LUN8
RICCOBONO A._1E_LUN8
SORCI A._1E_LUN8
ALLEGRA P._1E_LUN9
ARMETTA R._1E_LUN9
BONDI G._1E_LUN9
CAVADI A._1E_LUN9
COLOMBINO E._1E_LUN9
MACCATAIO S._1E_LUN9
MONTALBANO B._1E_LUN9
NICCHITTA D._1E_LUN9
PASSARO G._1E_LUN9
PATERNO A._1E_LUN9
RICCOBONO A._1E_LUN9
RICCOBONO A._1E_LUN9
SORCI A._1E_LUN9
ALLEGRA P._1E_LUN10
ARMETTA R._1E_LUN10
BONDI G._1E_LUN10
CAVADI A._1E_LUN10
COLOMBINO E._1E_LUN10
MACCATAIO S._1E_LUN10
MONTALBANO B._1E_LUN10
NICCHITTA D._1E_LUN10
PASSARO G._1E_LUN10
PATERNO A._1E_LUN10
RICCOBONO A._1E_LUN10
RICCOBONO A._1E_LUN10
SORCI A._1E_LUN10
ALLEGRA P._1E_LUN11
ARMETTA R._1E_LUN11
BONDI G._1E_LUN11
CAVADI A._1E_LUN11
COLOMBINO E._1E_LUN11
MACCATAIO S._1E_LUN11
MONTALBANO B._1E_LUN11
NICCHITTA D._1E_LUN11
PASSARO G._1E_LUN11
PATERNO A._1E_LUN11
RICCOBONO A._1E_